#### Setup

Be sure to setup the `OPENAI_API_KEY` environment variable before running this notebook.

In [3]:
import os 
import logging
import sys

# Setup OPENAI_API_KEY

os.environ["OPENAI_API_KEY"] = ""

# Setup logging

log = logging.getLogger(__name__)
logging.basicConfig(format="%(asctime)s | %(levelname)s | %(message)s", level=logging.INFO)

# Update sys.path (or use PYTHONPATH)

sys.path.insert(0, '..')

#### Read feedbacks

Let's read the feedback data scraped previously using the `scraping.ipynb` notebook.

In [4]:
import pandas as pd

df = pd.read_csv("./feedbacks.csv").dropna()

df.head()

,title,rating,text
0,This Sucker SUCKS!,5,"I just received this vacuum a few hours ago, b..."
1,"Worth the money, does what it says it will.",5,"My older vacuum was also a Bissell, a Bissell ..."
2,Best vacuum for the price!,5,"I am absolutely amazed with this vacuum, I’m a..."
3,Mostly Fantastic. Awesome performance.,5,My old vac broke. I bought a Shark but when it...
4,Extreme suction,5,I vacum at least 3 times a week if not more! I...


#### Filter & Sampling

For simplicity, we will keep only the feedbacks that have less than 300 chars and sample 15 items.

In [5]:
# Filter feedbacks text below 300 characters
df = df[df.text.str.len() < 300].reset_index(drop=True)

# Sample n feedbacks
df = df.sample(n=15).reset_index()

df.head()

,index,title,rating,text
0,37,Pretty good,5,I like deep cleaning and this vacuum allows th...
1,23,Great Vaccum!,5,This vacuum works exactly like I wanted it to....
2,22,Does the job,4,Very good budget vacuum. Cleans very well. Doe...
3,34,Bissell... Dog hair... Bissell WON!!!,5,It has great suction but when it started to su...
4,36,Great vaccum for the price,5,Such great suction on this vacuum. It manages ...


#### Analysis

Here is a simple analysis loop that will send the feedbacks to GPT-3 and parse the JSON response. The analysis could take some time, depending on the number of feedbacks and on OpenAI account type.

At the end, it updates the main dataframe with a new JSON column for each feedback and save the dataframe as CSV.

In [6]:
from random import choice
from tqdm.notebook import tqdm
from absa.analysis.gpt3 import analyze
from json import loads
from pprint import pprint
from textwrap import dedent

analysis_results = []
extra_prompts = []

logging.getLogger("openai").setLevel(logging.INFO)
logging.getLogger("requests").setLevel(logging.WARNING)

for i in tqdm(range(len(df)), desc="Analyzing reviews"):
    title = df.loc[i, "title"]
    text = df.loc[i, "text"]

    log.info(f"Analyzing feedback - \nTitle: {title}\nText: {text}\n")

    extra_prompt = choice(extra_prompts) if extra_prompts else ""

    res = analyze(
        text=text,
        extra_prompt="",
        max_tokens=1024,
        temperature=0.1,
        top_p=1,
    )

    raw_json = res["choices"][0]["text"].strip()

    try:
        json_data = loads(raw_json)
        analysis_results.append(json_data)

        log.debug(f"JSON response: {pprint(json_data)}")

        extra_prompts.append(f"\n{text}\n{raw_json}")
    except Exception as e:
        log.error(f"Failed to parse '{raw_json}' -> {e}")
        analysis_results.append([])

df["analysis"] = analysis_results
df.to_csv("./feedbacks_analysis.csv", index=False)


Analyzing reviews:   0%|          | 0/15 [00:00<?, ?it/s]

2022-09-08 22:47:55,840 | INFO | Analyzing feedback - 
Title: Pretty good
Text: I like deep cleaning and this vacuum allows that for sure. Has great suction and the attachments work well. It seems a little on the heavy side but it’s alright. Fluffs up the carpet nicely and easy to keep clean.

2022-09-08 22:47:55,843 | INFO | message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
2022-09-08 22:48:02,291 | INFO | message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=4730 request_id=ad5054e724247f6bb02d45c34e7805fc response_code=200
2022-09-08 22:48:02,297 | INFO | Analyzing feedback - 
Title: Great Vaccum!
Text: This vacuum works exactly like I wanted it to.  It picks up everything.

2022-09-08 22:48:02,298 | INFO | message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions


[{'aspect': 'Cleaning',
  'segment': 'I like deep cleaning and this vacuum allows that for sure',
  'sentiment': 'positive'},
 {'aspect': 'Suction', 'segment': 'Has great suction', 'sentiment': 'positive'},
 {'aspect': 'Attachments',
  'segment': 'the attachments work well',
  'sentiment': 'positive'},
 {'aspect': 'Weight',
  'segment': 'It seems a little on the heavy side but it’s alright',
  'sentiment': 'negative'},
 {'aspect': 'Carpet',
  'segment': 'Fluffs up the carpet nicely',
  'sentiment': 'positive'},
 {'aspect': 'Maintenance',
  'segment': 'easy to keep clean',
  'sentiment': 'positive'}]


2022-09-08 22:48:04,431 | INFO | message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=1926 request_id=ee884e90dee92d4ab56add630320503f response_code=200
2022-09-08 22:48:04,432 | INFO | Analyzing feedback - 
Title: Does the job
Text: Very good budget vacuum. Cleans very well. Does not fit under the couch and I did not think to get a retractable cord, I wish I had. But over all, its a very good vacuum cleaner. Sucked ups a cup of dirt in seconds that I robot left behind.

2022-09-08 22:48:04,433 | INFO | message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions


[{'aspect': 'Overall satisfaction',
  'segment': 'This vacuum works exactly like I wanted it to',
  'sentiment': 'positive'},
 {'aspect': 'Performance',
  'segment': 'It picks up everything',
  'sentiment': 'positive'}]


...

[{'aspect': 'Assembly', 'segment': 'Easy to assemble', 'sentiment': 'positive'},
 {'aspect': 'Emptying tank',
  'segment': 'emptying tank is straight forward',
  'sentiment': 'positive'},
 {'aspect': 'Suction', 'segment': 'awesome suction', 'sentiment': 'positive'},
 {'aspect': 'Longevity',
  'segment': 'Hopefully it will last',
  'sentiment': 'positive'}]


2022-09-08 22:48:57,581 | INFO | message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=2394 request_id=8adfac30789855f34e2301c02c41d747 response_code=200
2022-09-08 22:48:57,582 | INFO | Analyzing feedback - 
Title: Great vaccum for the price
Text: Such great suction on this vacuum. It manages to pickup so much first and hair from my carpets

2022-09-08 22:48:57,583 | INFO | message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions


[{'aspect': 'Overall satisfaction',
  'segment': 'This vacuum works exactly like I wanted it to',
  'sentiment': 'positive'},
 {'aspect': 'Performance',
  'segment': 'It picks up everything',
  'sentiment': 'positive'}]


2022-09-08 22:49:00,155 | INFO | message='OpenAI API response' path=https://api.openai.com/v1/completions processing_ms=2389 request_id=31adf1c51b1478a3f1106126b9b70139 response_code=200
2022-09-08 22:49:00,156 | ERROR | Failed to parse '.

[
  { "aspect": "Suction", "segment": "Such great suction on this vacuum", "sentiment": "positive" },
  { "aspect": "Performance", "segment": "It manages to pickup so much first and hair from my carpets", "sentiment": "positive" }
]' -> Expecting value: line 1 column 1 (char 0)
2022-09-08 22:49:00,157 | INFO | Analyzing feedback - 
Title: Picks up dog hair on all surfaces
Text: Good suction, picks up dog hair, dirt, hay, etc. Easy to empty canister. Easy to clean filter. Does get clogged, but easy to take apart and unclog, takes no time at all.

2022-09-08 22:49:00,157 | INFO | message='Request to OpenAI API' method=post path=https://api.openai.com/v1/completions
2022-09-08 22:49:06,852 | INFO | message='OpenAI API response' path=https://api.openai.

[{'aspect': 'Suction', 'segment': 'Good suction', 'sentiment': 'positive'},
 {'aspect': 'Picks up',
  'segment': 'picks up dog hair, dirt, hay, etc.',
  'sentiment': 'positive'},
 {'aspect': 'Canister',
  'segment': 'Easy to empty canister.',
  'sentiment': 'positive'},
 {'aspect': 'Filter',
  'segment': 'Easy to clean filter.',
  'sentiment': 'positive'},
 {'aspect': 'Clogging',
  'segment': 'Does get clogged, but easy to take apart and unclog, takes no '
             'time at all.',
  'sentiment': 'negative'}]


#### Playing with analysis results

Start from here if you want to play or display the analysis results.

In [7]:
import pandas as pd

df = pd.read_csv("./feedbacks_analysis.csv")

df

,index,title,rating,text,analysis
0,37,Pretty good,5,I like deep cleaning and this vacuum allows th...,"[{'aspect': 'Cleaning', 'segment': 'I like dee..."
1,23,Great Vaccum!,5,This vacuum works exactly like I wanted it to....,"[{'aspect': 'Overall satisfaction', 'segment':..."
2,22,Does the job,4,Very good budget vacuum. Cleans very well. Doe...,"[{'aspect': 'Overall satisfaction', 'segment':..."
3,34,Bissell... Dog hair... Bissell WON!!!,5,It has great suction but when it started to su...,"[{'aspect': 'Suction', 'segment': 'It has grea..."
4,36,Great vaccum for the price,5,Such great suction on this vacuum. It manages ...,[]
5,10,Great value,5,I love this. I have 2 dogs and it doesn't clo...,"[{'aspect': 'Overall satisfaction', 'segment':..."
6,32,Práctica y eficiente,4,"Trabaja muy bien, fácil de limpiar los filtros...","[{'aspect': 'Overall satisfaction', 'segment':..."
7,30,Excellent appliance,5,This vacuum is so great even the cleaning lady...,"[{'aspect': 'Overall satisfaction', 'segment':..."
8,11,From a Shark to a Bissell!!!,5,"I was a shark believer, and i still like their...","[{'aspect': 'Reliability', 'segment': 'after h..."
9,14,Práctica y eficiente,4,"Trabaja muy bien, fácil de limpiar los filtros...","[{'aspect': 'Overall satisfaction', 'segment':..."


In [8]:
from ast import literal_eval

df.analysis = df.analysis.apply(literal_eval)

analysis_results = df.analysis

analysis_results[:5]

0    [{'aspect': 'Cleaning', 'segment': 'I like dee...
1    [{'aspect': 'Overall satisfaction', 'segment':...
2    [{'aspect': 'Overall satisfaction', 'segment':...
3    [{'aspect': 'Suction', 'segment': 'It has grea...
4                                                   []
Name: analysis, dtype: object

In [14]:
annotations = []

for i, entry in enumerate(analysis_results):
    for a in entry:
        a["review_id"] = i
        annotations.append(a)

analysis_df = pd.DataFrame(annotations)

analysis_df.to_csv("./analysis.csv", index=False)

analysis_df

,aspect,segment,sentiment,review_id
0,Cleaning,I like deep cleaning and this vacuum allows th...,positive,0
1,Suction,Has great suction,positive,0
2,Attachments,the attachments work well,positive,0
3,Weight,It seems a little on the heavy side but it’s a...,negative,0
4,Carpet,Fluffs up the carpet nicely,positive,0
5,Maintenance,easy to keep clean,positive,0
6,Overall satisfaction,This vacuum works exactly like I wanted it to,positive,1
7,Performance,It picks up everything,positive,1
8,Overall satisfaction,Very good budget vacuum,positive,2
9,Cleaning,Cleans very well,positive,2


In [10]:
import plotly.express as px

fig = px.bar(
    analysis_df,
    x="aspect",
    color="sentiment",
    barmode="stack",
    color_discrete_map={
        "positive": "#52AC5E",
        "negative": "#e34a2d",
        "neutral": "gray",
    },
    title="Aspect vs Sentiment",
    template="plotly_white",
)

fig.show()


#### Display results in console

This will display the annotated feedbacks in the console.

In [11]:
from rich.console import Console

console = Console()

for i, review in enumerate(df.to_dict("records")):
    text = review["text"]

    try:

        for ann in analysis_results[i]:
            color = "green" if ann["sentiment"] == "positive" else "red"
            text = text.replace(
                ann["segment"],
                f" [bold white on {color}]{ann['segment']}[/bold white on {color}] ([orange1]{ann['aspect']}[/orange1])",
            )

        console.print(f"{review['title']}\n\n{text}")

    except Exception as e:
        print(f"Failed to parse {review['title']} {e}")
        continue


Pretty good

 I like deep cleaning and this vacuum allows that for sure (Cleaning).  Has great suction (Suction) and  the 
attachments work well (Attachments).  It seems a little on the heavy side but it’s alright (Weight).  Fluffs up the
carpet nicely (Carpet) and  easy to keep clean (Maintenance).

Great Vaccum!

 This vacuum works exactly like I wanted it to (Overall satisfaction).   It picks up everything (Performance).

Does the job

 Very good budget vacuum (Overall satisfaction).  Cleans very well (Cleaning).  Does not fit under the couch (Size)
and  I did not think to get a retractable cord, I wish I had (Cord). But over all, its a very good vacuum cleaner. 
Sucked ups a cup of dirt in seconds that I robot left behind (Comparison).

Bissell... Dog hair... Bissell WON!!!

 It has great suction (Suction) but when it started to suck my sheet up  it started to smoke a little (Smoking), 
luckily the sheet was very easy to remove. The smoke stopped and most like was from the friction of the sheet.

Great vaccum for the price

Such great suction on this vacuum. It manages to pickup so much first and hair from my carpets

...

I asked for a vacuum cleaner not sheet sucker

Works great on pet hair

Bissell CleanView Swivel Pet Reach Vacuum Cleaner

 Easy to assemble (Assembly), and  emptying tank is straight forward (Emptying tank). Used it to vacuum 1 room, and
it filled the tank twice. Very impressed,  awesome suction (Suction).  Hopefully it will last (Longevity), but so 
far-so good.

Great Vaccum!

 This vacuum works exactly like I wanted it to (Overall satisfaction).   It picks up everything (Performance).

Great vaccum for the price

Such great suction on this vacuum. It manages to pickup so much first and hair from my carpets

Picks up dog hair on all surfaces

 Good suction (Suction),  picks up dog hair, dirt, hay, etc. (Picks up)  Easy to empty canister. (Canister)  Easy 
to clean filter. (Filter)  Does get clogged, but easy to take apart and unclog, takes no time at all. (Clogging)

#### Display results in HTML

This will display the annotated feedbacks in a prettier way using HTML.

In [12]:
import re
from IPython.display import display, HTML
from html import escape

css = """
<style>
    .container {
        background-color: #fff;
        padding: 15px
    }

    p.feedback {
        margin-top: 5px;
        color: #595f6d;
        line-height: 2
    }

    h5.title {
        color: #b6bcc8;
        margin: 15px 0 0 0;
        padding: 0;
        font-style: italic;
    }

    .annotation {
        color: #777;
        padding: 2px;
        font-weight: bold !important;
        border-radius: 1px;
        border-bottom: 4px solid;
    }

    .aspect {
        color: #6eb2e7;
        padding-left: 10px;
        font-size: 12px;
    }
</style>
"""


def ireplace(text, old, new):
    pattern = re.compile(old, re.IGNORECASE)
    return pattern.sub(new, text)


html = f"{css}"

for i, review in enumerate(df.to_dict("records")):
    text = escape(review["text"])

    try:
        for ann in analysis_results[i]:
            color = "#2bbf6d" if ann["sentiment"] == "positive" else "#cf2a43"

            text = ireplace(
                text,
                ann["segment"],
                f"<span class='annotation' style='border-color: {color}'>{escape(ann['segment'])} <span class='aspect'>{ann['aspect']}</span></span>",
            )

        html += f"""

            <div class='container'>
                <h5 class='title'>{review['title']}</h5>
                <p class='feedback'>{text}</p>
            </div>
        """

    except Exception as e:
        print(f"Failed to parse {review['title']} {e}")
        continue

display(HTML(html))
